# ELYZA-tasks-100の評価 （cerebras APIの利用）

本サンプルは`Google Colab`(CPU) 環境での実行を想定して作成しております。

### 評価対象ファイル
ELYZA-tasks-100の`input`に対する応答出力をLLMで生成し、順番通りに`task_id`(0~99)と`output`キーに格納したjsonlファイルが評価対象となります。

jsonlファイルの作成については、GITリポジトリの`gen-01_tasks100.ipynb`を参考にしても良いですし、別のプログラムで出力したファイルを使っても構いません。

### 1. モジュールの導入

In [1]:
!pip install -U datasets

# cerebras 関連のモジュール
!pip install langchain-cerebras
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 26.0 MB/s eta 0:00:00
   ━━

### 2. Cerebras API のトークン設定

In [2]:
# Cerebras api Token を指定
CEREBRAS_API_KEY = "" #@param {type:"string"}

import os
os.environ["CEREBRAS_API_KEY"] = CEREBRAS_API_KEY



### 3. Githubリポジトリからツールのダウンロード


In [3]:
#リポジトリコード
!git clone  https://github.com/tana114/elyza-tasks-grader-like
#ディレクトリの移動
%cd /content/elyza-tasks-grader-like

Cloning into 'elyza-tasks-grader-like'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 2), reused 19 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 17.97 KiB | 5.99 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/elyza-tasks-grader-like


In [4]:
#ディレクトリの移動
%cd /content/elyza-tasks-grader-like
!ls

/content/elyza-tasks-grader-like
client	data  elyza_task  model  README.md  util


### 4. ツールを使用した評価

#### 評価対象ファイルのアップロード
ELYZA-tasks-100の`input`に対する応答出力をLLMで生成し、順番通りに`task_id`(0~99)と`output`キーに格納したjsonlファイルが評価対象となります。

ここでは、事前に生成しておいた評価対象ファイル`elyza100_gen.jsonl`を`/content`にアップロードした前提で進めます。

#### 評価に使用するLLMの準備

ここでは、cerebras API の "llama3.1-70b" モデルを指定しています。

In [ ]:
from model.cerebras_llm import CerebrasChatBase

llm_main = CerebrasChatBase(
    model_name="llama3.1-70b",  # 評価に使用するGroqのモデル
    requests_per_second=0.15,  # リクエストの時間間隔を調整（数値が小さいほうが時間間隔が長い）
    temperature=0.8,
)

#### ELYZA-kasks-100 の 'eval_aspect' に基づいた評価

評価結果を`score`として出力します。

`eval_batch_size`という引数がありますが、これが1リクエストで同時に処理する件数です。`1`が理想ですが、数を増やすことで1リクエストでの処理件数を稼げます。

In [9]:
from logging import DEBUG, INFO, WARN, ERROR, basicConfig

basicConfig(level=WARN)

from elyza_task.eval_manager import ElyzaTasksEvaluateManager

# elyza_jsonl = './data/test/elyza100_jp.jsonl'  # elyza-tasks-100を用いる場合は指定不要
em = ElyzaTasksEvaluateManager(
    main_llm=llm_main,
    eval_batch_size=5,  # 1回のリクエストで何個のデータを処理するか。数が増えると成功率と精度が下がると思います（1が望ましい）。
    # tasks_file_path=elyza_jsonl,
)

# 評価対象となるjosnl。ELYZA-tasks-100の順番に対応した'task_id'と'output'キーが必須。
target_jsonl = '/content/elyza100_gen.jsonl'

assert not os.path.isfile('/content/eval_elyza100_gen.jsonl'), '前回の評価結果が残っている場合は削除してください。'

score = em(eval_jsonl_path=target_jsonl)
print(score)  # 1.6

0it [00:00, ?it/s]

1.6


評価結果が`eval_{target_jsonl}`というファイル名で生成されるので、ローカルに保存します。

In [10]:
# ローカルに保存
from google.colab import files
files.download('/content/eval_elyza100_gen.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>